Univerzitet u Sarajevu

Elektrotehnički fakultet

#### **Dubinska analiza podataka**

# Laboratorijska vježba 5. - Sintetičko stvaranje podataka

---

Izrada laboratorijske vježbe vrši se u ovom *Jupyter Notebook*-u. Isti je potrebno konvertovati u PDF dokument i predati na email adresu ekrupalija1@etf.unsa.ba.

Ime i prezime studenta, broj indeksa:

Haris Masovic, 1689/17993

Datum izrade izvještaja:

11.04.2021

## Dependencies

In [1]:
import sys
!pip install nlpaug snorkel transformers

     |████████████████████████████████| 399kB 10.3MB/s 
     |████████████████████████████████| 153kB 22.9MB/s 
     |████████████████████████████████| 2.2MB 12.0MB/s 
     |████████████████████████████████| 1.8MB 57.5MB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 870kB 41.8MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556408 sha256=22372f85711fd45c3d964f04685b8918a4762fb62b6535507fbbf2ce485959fe
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=176fe606f65b7d9be252906ad4a788b2deddee0f379c49fbaf6f21c0caaac20b
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built networkx sacremoses
ERROR: tensorflow 2.4.1 has requirem

## Zadatak 1. - Zamjena stringa na nivou slova

Za sintetičku (vještačku) promjenu tekstualnih podataka koristeći tehnike prirodne obrade teksta (NLP) koristi se biblioteka <code> nlpaug </code> koju je potrebno instalirati preko konzole (<code> pip install nlpaug </code>). Osnovne informacije o ovoj biblioteci mogu se naći na [sljedećem linku](https://github.com/makcedward/nlpaug) i [sljedećem linku](https://nlpaug.readthedocs.io/en/latest/augmenter/augmenter.html).

Prvo je potrebno izvršiti import neophodnih biblioteka, što je izvršeno u nastavku.

In [2]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

# isključivanje prikaza upozorenja za deprecated funkcije
import warnings
warnings.filterwarnings('ignore')

Zamjena na nivou slova vrši se koristeći različite objekte iz biblioteke <code> nlpaug.augmenter.char </code>. U nastavku će se koristiti sljedeći objekti:

1. <code> nac.OcrAug() </code>, koji dodaje OCR grešku u postojeći string;

2. <code> nac.KeyboardAug() </code>, koji mijenja slova slovima koja imaju blizak ASCII broj;

3. <code> nac.RandomCharAug(action = x) </code>, koji vrši specificiranu akciju bez zakonitosti, odnosno nad nasumičnim karakterima stringa. <code> x </code> je potrebno zamijeniti akcijom koja se želi izvršiti (<code> 'insert' </code>, <code> 'substitute' </code>, <code> 'swap' </code> ili <code> 'delete' </code>).

Nad kreiranim objektom za zamjenu slova može se primijeniti funkcija <code> augment </code> koja kao parametar prima string nad kojim se vrši zamjena na nivou slova. Kao rezultat se dobiva modificirani string.

**Zadatak:**
    
> Definisati string 'Dubinska analiza podataka', a zatim nad njim izvršiti zamjenu na nivou slova koristeći sve prethodno opisane objekte (OCR, ASCII i četiri nasumična augmentatora). Prikazati originalni i sve modificirane stringove.
>
>> **Uputstvo:**
>>
>> Sve rezultujuće stringove smjestiti u zasebne varijable, jer će ih biti neophodno koristiti u sljedećem dijelu zadatka. Paziti da se ne pobriše vrijednost originalnog stringa (iz istog razloga).

In [43]:
sentence = 'Dubinska analiza podataka'

processor = nac.OcrAug()
ocr = processor.augment(sentence)

processor = nac.KeyboardAug()
keyboard = processor.augment(sentence)

processor = nac.RandomCharAug(action = 'insert')
insert = processor.augment(sentence)

processor = nac.RandomCharAug(action = 'substitute')
substitute = processor.augment(sentence)

processor = nac.RandomCharAug(action = 'swap')
swap = processor.augment(sentence)

processor = nac.RandomCharAug(action = 'delete')
delete = processor.augment(sentence)

print(*['Orginal tekst -', sentence])
print(*['Augmentacija ocr -', ocr])
print(*['Augmentacija keyboard -', keyboard])
print(*['Augmentacija insert -', insert])
print(*['Augmentacija substitute -', substitute])
print(*['Augmentacija swap -', swap])
print(*['Augmentacija delete -', delete])


Orginal tekst - Dubinska analiza podataka
Augmentacija ocr - Dubinska analiza p0datara
Augmentacija keyboard - Dubinska ana<ixa podataka
Augmentacija insert - Dubinska danalUiza podataka
Augmentacija substitute - Dubinskz analiza podataka
Augmentacija swap - Dunbiska analiza podataka
Augmentacija delete - Dubinska anlza podataka


**Zadatak:**
> Izračunati udaljenost rezultnih stringova od originalnog stringa. Ispisati rezultate i analizirati koja metoda je dala najveću različitost.
>
>> **Uputstvo:**
>> Minimalna udaljenost je 0%, a maksimalna 100% i računa se prema formuli: (broj karaktera koji su na korespondentnom mjestu u stringu različiti) / (ukupan broj karaktera) * 100.
>>
>> Korespondentno mjesto označava istu poziciju karaktera u stringu. Npr. udaljenost između riječi 'dubina' i 'd0bina' je 1, jer se samo karakteri na poziciji 1 ('u' i '0') razlikuju.

In [44]:
import numpy as np

distances = np.array([0, 0, 0, 0, 0, 0])

sentence_length = len(sentence)

for i in range(0, sentence_length):
    if i >= len(ocr) or sentence[i] != ocr[i]:
        distances[0] += 1
    if i >= len(keyboard) or sentence[i] != keyboard[i]:
        distances[1] += 1
    if i >= len(insert) or sentence[i] != insert[i]:
        distances[2] += 1
    if i >= len(substitute) or sentence[i] != substitute[i]:
        distances[3] += 1
    if i >= len(swap) or sentence[i] != swap[i]:
        distances[4] += 1
    if i >= len(delete) or sentence[i] != delete[i]:
        distances[5] += 1
        
        
distances = distances / sentence_length * 100

print('Udaljenosti:')

for i in distances:
    print(*[i, '%.'])

Udaljenosti:
8.0 %.
8.0 %.
56.00000000000001 %.
4.0 %.
12.0 %.
48.0 %.


- Najvecu razlicitost je dala `insert` metoda, mada u zavisnosti slucaja moze biti i `delete` metoda, jer ovisi se od indexa pozicije zamjene odnosno brisanja karaktera.

**Zadatak:**

> Tri puta generisati po tri nasumične zamjene na nivou slova po izboru. Analizirati njihovu prosječnu udaljenost od originalnog stringa - da li je približno jednaka?
>
>> **Uputstvo:**
>>
>> Za generisanje više zamjena odjednom, pri pozivu funkcije <code> augment </code> dodaje se parametar <code> n = x </code>, pri čemu je <code> x </code> broj stringova koji se žele generisati. Rezultni stringovi su sada u formi liste.
>>
>> Prosječnu udaljenost za jednu listu izračunati po formuli: (suma udaljenosti svih stringova iz liste) / (broj stringova u listi).

In [47]:
loop_time = 3

for loop in range(0, loop_time):
    number_of_swaps = 3
    
    processor = nac.RandomCharAug(action = 'swap')
    swaps = processor.augment(sentence, n = number_of_swaps)
    
    distances_single_loop = np.array([0,0,0])
    
    for i in range(0, number_of_swaps):
        swap = swaps[i]
        len_swap = len(swap)
        
        for j in range(0, sentence_length):
            if j >= len_swap or sentence[j] != swap[j]:
                distances_single_loop[i] += 1
    
    distances_single_loop = distances_single_loop / sentence_length * 100
    print(*['Average udaljenost pokusaja', loop + 1, '-', sum(distances_single_loop) / 3, '%.'])

Average udaljenost pokusaja 1 - 16.0 %.
Average udaljenost pokusaja 2 - 13.333333333333334 %.
Average udaljenost pokusaja 3 - 16.0 %.


- Prosjecna udaljenost od orginalnog stringa je priblizno jednaka.

Za sintetičko kreiranje podataka koristi se i biblioteka <code> snorkel </code>, koju je također potrebno instalirati preko konzole (<code> pip install snorkel </code>). Voditi računa da je instalacija ove biblioteke kompleksniji postupak (zahtijeva instaliran modul Cython i Microsoft Visual C++ 14.0). Sve informacije o korištenju biblioteke mogu se naći na [sljedećem linku](https://buildmedia.readthedocs.org/media/pdf/snorkel/latest/snorkel.pdf).

*Snorkel* za augmentaciju koristi dva modula:

1. <code> ApplyOnePolicy </code>, objekat koji definiše postavke za sintetičko stvaranje riječi.

2. <code> TFApplier </code>, objekat koji vrši sintetičko stvaranje. Ovaj objekat prima dva parametra: listu funkcija koje se žele primijeniti na objekat i prethodno kreiran objekat postavki augmentacije podataka.

U tu svrhu potrebno je definisati transformacijsku funkciju (TF) koja će se primijeniti na podatke. U kodu ispod definisana je funkcija koja po principu nasumičnosti vrši zamjenu karaktera iz rečenice nekim drugim karakterom.

Nakon definisanja modela za augmentaciju podataka, nad modelom je moguće primijeniti funkciju <code> apply </code>, koja kao parametar prima listu podataka nad kojima se želi primijeniti, a vraća istu listu sa novim redom transformisanih podataka.

In [6]:
import random
import string
import numpy as np
from snorkel.augmentation import ApplyOnePolicy, TFApplier
import random
import nltk
from nltk.corpus import wordnet as wn
from snorkel.augmentation import transformation_function

def replace_token(doc, idx, replacement):
    doc = doc[0 : idx] + replacement + doc[idx + 1 :]
    return doc

@transformation_function()
def replace_character_in_word(x):
    for i in range(0, (int)(len(x) / 10)):
        index = random.randint(0, len(x) - 1)
        character = random.choice(string.ascii_lowercase)
        x = replace_token(x, index, character)
    return x

**Zadatak:**

> Definisati model za augmentaciju i izvršiti prethodno definisanu funkciju augmentacije nad rečenicom "Natural Language Processing Is Amazing".
>
>> **Uputstvo:**
>>
>> Prvo je potrebno definisati postavke, a zatim model augmentacije. U kodu iznad izvršen je import neophodnih klasa objekata.
>>
>> Model kao prvi parametar prima listu koja treba sadržati transformacijsku funkciju.

In [7]:
aug_sentence = ['Natural Language Processing Is Amazing']

policy = ApplyOnePolicy()
applier = TFApplier([replace_character_in_word], policy)

after_aug_sentence = applier.apply(aug_sentence)

print(*['Recenica prije augmentacije:', aug_sentence[0]])
print(*['Recenica nakon augmentacije:', after_aug_sentence[1]])

100%|██████████| 1/1 [00:00<00:00, 942.54it/s]

Recenica prije augmentacije: Natural Language Processing Is Amazing
Recenica nakon augmentacije: Namural Langubge Processing Is Amaling


**Zadatak:**

> Samostalno definisati funkciju koja izvršava zamjenu karaktera u riječi na sličan način kao funkcije iz prvog dijela zadatka (po želji). Izvršiti augmentaciju podataka i usporediti dobivene rezultate.

In [8]:
@transformation_function()
def swap_random_three_values(x):
    for i in range(0, (int)(len(x) / 15)):
        index = random.randint(0, len(x) - 1)
        second_index = random.randint(0, len(x) - 1)
        third_index = random.randint(0, len(x) - 1)

        first_value = x[index]
        second_value = x[second_index]
        third_value = x[third_index]
        
        x = x[0 : index] + third_value + x[index + 1 :]
        x = x[0 : second_index] + first_value + x[second_index + 1 :]
        x = x[0 : third_index] + second_value + x[third_index + 1 :]

    return x

aug_sentence = ['Natural Language Processing Is Amazing']

policy = ApplyOnePolicy()
applier = TFApplier([swap_random_three_values], policy)

after_aug_sentence = applier.apply(aug_sentence)

print(*['Recenica prije augmentacije:', aug_sentence[0]])
print(*['Recenica nakon augmentacije:', after_aug_sentence[1]])

100%|██████████| 1/1 [00:00<00:00, 2945.44it/s]

Recenica prije augmentacije: Natural Language Processing Is Amazing
Recenica nakon augmentacije: Natug lsLanguage Proceasinr Is Amazing


- `swap_random_three_values` uzima tri slucajna karaktera i mijenja ih ciklicno u stringu. Vidimo da duzina ostaje ista, samo se mijenjaju 3 karaktera.

---

## Zadatak 2. - Zamjena stringa na nivou riječi

Zamjena na nivou riječi vrši se koristeći različite objekte iz biblioteke <code> nlpaug.augmenter.word </code>. U nastavku će se koristiti sljedeći objekti:

1. <code> naw.SpellingAug() </code>, koji vrši zamjenu riječi koristeći greške u spelovanju;

2. <code> naw.ContextualWordEmbsAug() </code>, koji vrši zamjenu riječi na osnovu kontekstualne sličnosti, pritom koristeći jedan od šest dostupnih algoritama (npr. <code> bert-base-uncased </code>, <code> distilbert-base-uncased </code> ili <code> roberta-base </code>). Za korištenje objekta ovog tipa potrebno je instalirati dodatnu biblioteku: <code> pip install transformers </code>;

3. <code> naw.AntonymAug() </code>, koji vrši zamjenu riječi na osnovu antonima;

4. <code> naw.SynonymAug() </code>, koji vrši zamjenu riječi na osnovu sinonima registrovanih u nekoj od dostupnih baza riječi (<code> 'wordnet' </code> ili <code> 'ppdb' </code>);

5. <code> naw.RandomWordAug() </code>, koji vrši zamjenu mjesta postojećih riječi koristeći nasumičnu preraspodjelu.

Nad kreiranim objektom za zamjenu na nivou riječi može se primijeniti funkcija <code> augment </code>, na isti način kao i u prethodnom zadatku.

**Zadatak:**
    
> Definisati string 'DAP is one of the subjects of the final semester at the Faculty of Electrical Engineering.', a zatim nad njim izvršiti zamjenu na nivou riječi koristeći sve prethodno opisane objekte. Prikazati originalni i sve modificirane stringove.
>
>> **Uputstvo:**
>>
>> Sve rezultujuće stringove smjestiti u zasebne varijable, jer će ih biti neophodno koristiti u sljedećem dijelu zadatka. Paziti da se ne pobriše vrijednost originalnog stringa (iz istog razloga).
>>
>> naw.ContextualWordEmbsAug() prima jedan parametar: <code> model_path </code>, koji se postavlja na jedan od postojećih algoritama.
>>
>> naw.SynonymAug() prima jedan parametar: <code> model_path </code>, koji se postavlja na jednu od dostupnih baza podataka.
>>
>> naw.RandomWordAug() prima jedan parametar: <code> action </code>, koji se postavlja na <code> 'swap' </code>.

In [22]:
import nlpaug.augmenter.word as naw

sentence = "DAP is one of the subjects of the final semester at the Faculty of Electrical Engineering."

processor = naw.SpellingAug()
spelling = processor.augment(sentence)

processor = naw.ContextualWordEmbsAug(model_path = "bert-base-uncased")
context = processor.augment(sentence)

processor = naw.AntonymAug()
antonyms = processor.augment(sentence)

processor = naw.SynonymAug(aug_src='wordnet')
synonims = processor.augment(sentence)

processor = naw.RandomWordAug(action = 'swap')
random_swap = processor.augment(sentence)

print(*['Orginal tekst -', sentence])
print(*['Augmentacija spelling -', spelling])
print(*['Augmentacija context -', context])
print(*['Augmentacija antonimi -', antonyms])
print(*['Augmentacija synonims -', synonims])
print(*['Augmentacija random swap -', random_swap])


Orginal tekst - DAP is one of the subjects of the final semester at the Faculty of Electrical Engineering.
Augmentacija spelling - DAP is 1one of tehere sobjects of the finel semester at the Facolty of Electrical Engineering.
Augmentacija context - dap selection one by the subjects of the final program at the faculty at environmental engineering.
Augmentacija antonimi - DAP differ one of the subjects of the final semester at the Faculty of Electrical Engineering.
Augmentacija synonims - DAP constitute unitary of the field of study of the final semester at the Faculty of Electrical Engineering.
Augmentacija random swap - is DAP one of the subjects of the semester final the Faculty at Electrical of Engineering.


**Zadatak:**
> Izračunati udaljenost rezultnih stringova od originalnog stringa. Ispisati rezultate i analizirati koja metoda je dala najveću različitost.
>
>> **Uputstvo:**
>> Minimalna udaljenost je 0%, a maksimalna 100% i računa se prema formuli: (broj riječi koje su na korespondentnom mjestu u stringu različite) / (ukupan broj riječi) * 100.
>>
>> Korespondentno mjesto označava isti broj riječi u stringu. Npr. udaljenost između rečenica 'dubina nije' i 'dubina je' je 1, jer se samo riječi na poziciji 1 ('nije' i 'je') razlikuju.
>>
>> Za razdvajanje stringa u listu riječi može se koristiti funkcija <code> split </code> sa razmakom kao delimiterom.

In [23]:
distances = np.array([0, 0, 0, 0, 0])

sentence_length = len(sentence)
sentence_w = sentence.split(" ")

spelling_w = spelling.split(" ")
context_w = context.split(" ")
antonyms_w = antonyms.split(" ")
synonims_w = synonims.split(" ")
random_w = random_swap.split(" ")

number_words = len(sentence_w)

for i in range(0, number_words):
    if sentence_w[i] != spelling_w[i]:
        distances[0] += 1
    if sentence_w[i] != context_w[i]:
        distances[1] += 1
    if sentence_w[i] != antonyms_w[i]:
        distances[2] += 1
    if sentence_w[i] != synonims_w[i]:
        distances[3] += 1
    if sentence_w[i] != random_w[i]:
        distances[4] += 1

        
distances = distances / number_words * 100

print('Udaljenosti:')

for i in distances:
    print(*[i, '%.'])

Udaljenosti:
31.25 %.
50.0 %.
6.25 %.
75.0 %.
56.25 %.


- Najvecu razlicitost u vecini slucajeva je dala `SynonymAug` metoda

I za zamjenu na nivou riječi koristi se <code> snorkel </code> biblioteka, na isti način kao i u prethodnom zadatku. U tu svrhu koriste se isti objekti (model za augmentaciju i postavke), ali je struktura transformacijskih funkcija drukčija.
    
U nastavku je prikazan kod kojim se definiše transformacijska funkcija (TF) za zamjenu riječi u rečenici sa sinonimom iz baze podataka *WordNet*.

In [24]:
import random
import nltk
from nltk.corpus import wordnet as wn
from snorkel.augmentation import transformation_function
from snorkel.augmentation import ApplyOnePolicy, TFApplier

def get_synonyms(word):
    lemmas = set().union(*[s.lemmas() for s in wn.synsets(word)])
    return list(set(l.name().lower().replace("_", " ") for l in lemmas) - {word})


@transformation_function()
def tf_replace_word_with_synonym(x):
    words = x.lower().split()
    idx = random.choice(range(len(words)))
    synonyms = get_synonyms(words[idx])
    if len(synonyms) > 0:
        x = " ".join(words[:idx] + [synonyms[0]] + words[idx + 1 :])
        return x

**Zadatak:**

> Zamijeniti riječi u rečenici "Natural Language Processing Is Amazing" sa sinonimima. Generisati pet takvih rečenica.
>
>> **Uputstvo:**
>>
>> Kreirati objekte na isti način kao u prethodnom zadatku, s tim što je pri kreiranju objekta postavki <code> ApplyOnePolicy </code> potrebno dodati parametar <code> n_per_original </code> kojim se definiše broj izlaznih rečenica.

In [27]:
aug_sentence = ['Natural Language Processing Is Amazing']

policy = ApplyOnePolicy(n_per_original = 5, keep_original = True)
applier = TFApplier([tf_replace_word_with_synonym], policy)

after_aug_sentences = applier.apply(aug_sentence)

print(*['Recenica prije augmentacije:', aug_sentence[0]])
print('Recenice nakon augmentacije:')

for i in range(1, len(after_aug_sentences)):
  print(after_aug_sentence[i])

100%|██████████| 1/1 [00:00<00:00, 119.46it/s]

Recenica prije augmentacije: Natural Language Processing Is Amazing
Recenice nakon augmentacije:
natural language processing is awful
natural voice communication processing is amazing
innate language processing is amazing
natural voice communication processing is amazing
natural language processing exist amazing


---

## Zadatak 3. - Zamjena stringa na nivou rečenice

Zamjena na nivou rečenice vrši se koristeći različite objekte iz biblioteke <code> nlpaug.augmenter.sentence </code>. U nastavku će se koristiti sljedeći objekat:

<code> nas.ContextualWordEmbsForSentenceAug() </code>, koji vrši dodavanje nove rečenice na osnovu kontekstualnog značenja izvornog stringa. Ovaj objekat kao parametar prima <code> model_path </code>, koji je moguće postaviti na jednu od tri vrijednosti: <code> xlnet-base-cased </code>, <code> gpt2 </code> ili <code> distilgpt2 </code>.

Nad kreiranim objektom za zamjenu na nivou rečenice može se primijeniti funkcija <code> augment </code>, na isti način kao i u prethodnom zadatku.

**Zadatak:**

> Definisati rečenicu 'Studying hard pays off because if you study hard you will gain the best grade.' a zatim nad njom kreirati po tri nove rečenice koristeći sva tri dostupna algoritma za generisanje novih rečenica na osnovu konteksta.
>
>> **Uputstvo:**
>>
>> Pri prvom pokretanju za svaki novi algoritam potrebno je preuzeti neophodne *file*-ove, što može potrajati nekoliko minuta.
>>
>> Za definisanje broja rečenica koje se žele generisati može se koristiti parametar <code> n </code> u funkciji <code> augment </code>, na isti način kao i u prethodnim zadacima.

In [29]:
sentence = "Studying hard pays off because if you study hard you will gain the best grade"

processor = nas.ContextualWordEmbsForSentenceAug(model_path = 'xlnet-base-cased')
xlnet = processor.augment(sentence, n = 3)

processor = nas.ContextualWordEmbsForSentenceAug(model_path = 'gpt2')
gpt2 = processor.augment(sentence, n = 3)

processor = nas.ContextualWordEmbsForSentenceAug(model_path = 'distilgpt2')
distilgpt2 = processor.augment(sentence, n = 3)

print(*['Original:', sentence])

print('xlnet algoritam:')
for i in xlnet:
  print(i)

print('gpt2 algoritam:')
for i in gpt2:
  print(i)

print('distilgpt2 algoritam:')
for i in distilgpt2:
  print(i)



Original: Studying hard pays off because if you study hard you will gain the best grade
xlnet algoritam:
Studying hard pays off because if you study hard you will gain the best grade at a school this subject offers!
Studying hard pays off because if you study hard you will gain the best grade when gaining computer classes like C-PS classes For online classes.
Studying hard pays off because if you study hard you will gain the best grade at all events of your life during that course!
gpt2 algoritam:
Studying hard pays off because if you study hard you will gain the best grade , i ion ( el in es : ies ( ) at / ic , t , as s , and ah 3 , s n 1 i on .
Studying hard pays off because if you study hard you will gain the best grade ia es d for ar ing s I es s ing at - in - , at it and was an for d ing , er - j The .
Studying hard pays off because if you study hard you will gain the best grade on I r io r o g ar _ er ar is es ig I ol i al - _ ) ik io A o i with , in es
distilgpt2 algoritam:
Stud

Analizirati prethodno generisane rečenice. Da li neke od njih nisu kontekstualno slične originalnoj rečenici? Da li se može odrediti na osnovu čega (kojeg dijela rečenice) su generisane baš takve rečenice?

- Neke od generisanih recenica nisu kontekstualno slicne recenici
- Moze, na osnovu random karaktera ili na osnovu jendnostavnosti da se bas ne uklapaju u smisao recenice, ali gledajuci u vidu konteksta, tu se nalaze.

**Zadatak:**

> Promijeniti postavke parametara u nastojanju da se postignu bolji rezultati.
>
>> **Uputstvo:**
>>
>> Koristiti isključivo <code> xlnet-base-cased </code> model. Promijeniti parametar <code> temperature </code> na 0.5 i 0.01. Promijeniti parametar <code> top_k </code> na 10, 200, 500 i 1000.

In [30]:
sentence = "Studying hard pays off because if you study hard you will gain the best grade"

temperatures = [0.5, 0.01]
top_k = [10, 200, 500, 1000]

for temp in temperatures:
  for k in top_k:
    processor = nas.ContextualWordEmbsForSentenceAug(model_path = 'xlnet-base-cased', temperature = temp, top_k = k)
    xlnet = processor.augment(sentence, n = 3)
    print(*['Augmentacija nad temperaturom:', temp, 'i k:', k])
    for i in xlnet:
      print(i)


Augmentacija nad temperaturom: 0.5 i k: 10
Studying hard pays off because if you study hard you will gain the best grade in math science or science writing skills in the college of you choice at age 13 through 17 without even knowing your major.
Studying hard pays off because if you study hard you will gain the best grade of your college.
Studying hard pays off because if you study hard you will gain the best grade in every major in college you ever took in your first 10 years!
Augmentacija nad temperaturom: 0.5 i k: 200
Studying hard pays off because if you study hard you will gain the best grade possible with the exam of this exam.
Studying hard pays off because if you study hard you will gain the best grade for every single single project.
Studying hard pays off because if you study hard you will gain the best grade from each subject that is tested to 100% on every final exam and homework every final homework you get through all exam for the first time of the exam every
Augmentacija

Analizirati prethodno generisane rečenice. Koja kombinacija parametara je dala najbolje, a koja najgore rezultate? Šta se iz toga može zaključiti o važnosti parametara <code> temperature </code> i <code> top_k </code>?

- Smanjenje temperature utice na smanjenje random mogucih dobijenih rijeci, i sto manja temperatura vise je logicniji odgovor u skladu pocetne recenice. Najbolji rezultati su dobijeni za `temp=0.01`
- Sto veci `top_k` znaci vise tokena koji mogu prouzrokovati drugacije rijeci, najbolji rezultat je dobijen za `top_k=500`.

---

## Zadatak 4. - Automatsko generisanje kratkog pregleda

Najviši nivo na kojem se može vršiti sintetičko kreiranje tekstualnih podataka je nivo diskursa. U tu svrhu koristi se isti modul kao i u prethodnom zadatku (<code> nlpaug.augmenter.sentence </code>). U nastavku će se koristiti sljedeći objekat:

<code> nas.AbstSummAug() </code>, koji kao parametar prima <code> model_path </code> za vrstu algoritma za transformaciju teksta (<code> facebook/bart-large-cnn </code>, <code> t5-small </code>, <code> t5-base </code> ili <code> t5-large </code>).

Nad kreiranim objektom za kreiranje kratkog pregleda može se primijeniti funkcija <code> augment </code>, na isti način kao i u prethodnom zadatku.

**Zadatak:**

> Kreirati string koji se sastoji od najmanje pet rečenica na engleskom jeziku po vlastitom izboru. Zatim kreirati kratke preglede koristeći prethodno definisane algoritme t5-small i t5-base (preostala dva algoritma su previše vremenski i memorijski zahtjevna). Analizirati koji od kratkih pregleda je najreprezentativniji.
>
>> **Uputstvo:**
>>
>> Za definisanje teksta sa više redova u kodu koristiti trostruke navodnike (""") umjesto jednostrukih (").
>>
>> Preuzimanje svih neophodnih *file*-ova za modele može potrajati nekoliko minuta.

In [34]:
five_sentences = """
If you're visiting this page, you're likely here because you're searching for a random sentence. 
Sometimes a random word just isn't enough, and that is where the random sentence generator comes into play. 
By inputting the desired number, you can make a list of as many random sentences as you want or need. 
Producing random sentences can be helpful in a number of different ways.
Feel free to use any of the random sentences for any project that you may be doing.
"""

processor = nas.AbstSummAug(model_path = 't5-small')
view = processor.augment(five_sentences)

print(*['t5-small view:', view])

processor = nas.AbstSummAug(model_path = 't5-base')
view = processor.augment(five_sentences)

print(*['t5-base view:', view])



t5-small view: the random sentence generator comes into play. make a list of as many random sentences as you want or need.
t5-base view: the random sentence generator can help you create a list of as many random sentences as you want. use the random sentences for any project that you may be doing.


- Najreprezentavniji predstavlja `t5-small`, jer je jednostavniji i konkretniji pregled.

**Zadatak:**

> Promijeniti postavke parametara u nastojanju da se postignu bolji rezultati.
>
>> Koristiti isključivo jedan model koji se odredi kao najtačniji u prethodnom dijelu zadatka.
>>
>> Pri kreiranju objekta dodati parametar <code> num_beam </code>. Postaviti njegovu vrijednost na 1, 5 i 10.

In [41]:
beams = [1, 5, 10]

for beam in beams:
  processor = nas.AbstSummAug(model_path = 't5-small', num_beam = beam)
  view = processor.augment(five_sentences)

  print(*['View t5-small, num_beam =', beam, '-', view])

View t5-small, num_beam = 1 - random sentences can be useful in a number of different ways. create random sentences by inputting the desired number.
View t5-small, num_beam = 5 - the random sentence generator comes into play. make a list of as many random sentences as you want or need.
View t5-small, num_beam = 10 - the random sentence generator comes into play. make a list of as many random sentences as you want or need.


Analizirati prethodno generisane kratke preglede. Koja kombinacija parametara je dala najbolje, a koja najgore rezultate? Šta se iz toga može zaključiti o važnosti parametara <code> num_beam </code> ?

- Veci `num_bean` rezultira da ce se uzimati vise rijeci i za svaku rijec pokusati generisati vise recenica. Veci `num_bean` znaci vise koristenje resura ali razarada vise varijacija za najbolje rijeci po vjerovatnoci odnosno formiranje recenice po najvjerovatnijim konteksnim rijecima. Najbolji rezultat daje `num_bean=5` jer `num_bean=10` daje isti rezultat, ali koristi manje resursa.

